# Easy21 assignment 
from UCL Course on RL https://www.davidsilver.uk/teaching/

full description https://www.davidsilver.uk/wp-content/uploads/2020/03/Easy21-Johannes.pdf

## 1  Implementation of Easy21 (10 marks)

You should write an environment that implements the game Easy21.

Specifically, write a function, namedstep, which takes as input a states (dealer’s firstcard 1–10 and the player’s sum 1–21), and an action $a$ (hit or stick), and returns a sample of the next state $s^′$(which may be terminal if the game is finished) and reward $r$.  We will be using this environment for model-free reinforcement learning, and you should not explicitly represent the transition matrix for the MDP. There is no discounting $(γ= 1)$.  You should treat the dealer’s moves as part of the environment, i.e.  calling step with a stick action will play out the dealer’s cards and return the final reward and terminal state.

1. first step two black cards to everybody
2. then player is turning
3. state = {dealer_sum, player_sum}
4. player does actions {hit,stick}
5. stick - no new card, hit - a new one
6. if player.sum() > 21 | < 1 then 'goes bust', return reward -1
7. if player stick the dealer starts making turns
8. the dealer always sticks on any sum of >=17 and hits otherwise
9. red card subtract value from player.sum, black card add
10. if the dealer 'goes bust' reward +1, if player -1, if player has 21 reward is 0

    


In [276]:
import random

class Easy21:
    def __init__(self):        
        self.dealer_sum = 0
        self.player_sum = 0
        self.started = False
                    
    def draw_a_card(self, black_only=False):                        
        if black_only: 
            r = random.randint(1, 10)                
        else:
            r = random.randint(1, 10)*(-1 if random.random() < 1/3 else 1)
        print('C:', r)
        return r
    
    def state(self):
            return [self.dealer_sum, self.player_sum]
        
    def draw(self):
        return self.player_sum == 21
    
    def goes_bust(self, summ):        
        if summ > 21 or summ < 1:            
            return True
        
    def dealer_hit(self):
        if self.dealer_sum < 17:
            return True
        return False
                
    def step(self, state, action): 
        if self.started == False:            
            self.player_sum = self.draw_a_card(True)
            self.dealer_sum = self.draw_a_card(True)
            self.started = True
            return self.state()
        else:
            if action == 'hit':
                self.player_sum += self.draw_a_card()
                if self.draw():
                    self.started = False
                    return 0
                if self.goes_bust(self.player_sum):
                    self.started = False
                    return -1
                return self.state()
            else:             
                while self.dealer_hit():
                    self.dealer_sum += self.draw_a_card()
                    if self.goes_bust(self.dealer_sum):
                        self.started = False
                        return 1
                if self.dealer_sum > self.player_sum:
                    return -1            
                return self.state()

e = Easy21() 

In [277]:
print(e.step(e.state(), 'hit'))
# print(e.state())

C: 4
C: 4
[4, 4]


In [278]:
print(e.step(e.state(), 'stick'))

C: 3
C: 8
C: -7
C: 8
C: -4
C: 10
1


## 2  Monte-Carlo Control in Easy21 (15 marks)

Apply Monte-Carlo control to Easy21.  
* Initialise the value function to zero.  
* Use a time-varying scalar step-size of $\alpha_t = {1}/{N(s_t, a_t)}$  and an $\epsilon$-greedy exploration strategy  with $\epsilon = N_0/(N_0+N(s_t))$,  where $N_0 =  100$  is  a  constant, $N(s)$  is the number of times that states has been visited, and $N(s,a)$ is the number of times that action $a$ has been selected from states.  
* Feel free to choose an alternative value for $N_0$, if it helps producing better results. 
* Plot the optimal value function $V^∗(s) = max_a{Q^∗(s,a)}$ using similar axes to the following figure taken from Sutton and Barto’s Blackjack example.

## 3 TD Learning in Easy21 (15 marks)

* Implement $Sarsa(λ)$ in 21s.  
* Initialise the value function to zero.  
* Use the samestep-size and exploration schedules as in the previous section. 
* Run the algorithmwith parameter values $\lambda \in \{ 0,0.1,0.2 , \ldots , 1 \}$.  
* Stop each run after 1000 episodes and report the mean-squared error $\sum _ { s , a } ( Q ( s , a ) - Q ^ { * } ( s , a ) ) ^ { 2 }$ over all states $s$ and  actions $a$,  comparing  the  true  values $Q^∗(s,a)$  computed  in  the  previous section with the estimated values $Q(s,a)$ computed by Sarsa.  
* Plot the mean-squared error against $\lambda$.  
* For $\lambda = 0$ and $\lambda = 1$ only, plot the learning curve ofmean-squared error against episode number.

## 4 Linear Function Approximation in Easy21 (15 marks)

We now consider a simple value function approximator using coarse coding.  
* Use a binary feature vector $\varphi(s,a)$ with 3∗6∗2 = 36 features.  
* Each binary feature has a value of 1 iff (s,a) lies within the cuboid of state-space corresponding to that feature, and the action corresponding to that feature.  The cuboids have the following overlapping intervals:

$ dealer(s) ={[1,4],[4,7],[7,10]}$

$ player(s) ={[1,6],[4,9],[7,12],[10,15],[13,18],[16,21]}$ 

$a={hit,stick}$

where
    * dealer(s) is the value of the dealer’s first card (1–10)
    * sum(s) is the sum of the player’s cards (1–21)
    
* Repeat the $Sarsa(λ)$ experiment from the previous section, but using linear value function approximation 
$Q ( s , a ) = \phi ( s , a ) ^ { T } \theta$.  Use a constant exploration of $\epsilon = 0.05$ and a constant step-size of 0.01.  
* Plot the mean-squared error againstλ.  For $\lambda= 0$ and $\lambda = 1$ only,  plot the learning curve of mean-squared erroragainst episode number.

## 5  Discussion

Discuss the choice of algorithm used in the previous section.
•What are the pros and cons of bootstrapping in Easy21?
•Would  you  expect  bootstrapping  to  help  more  in blackjack or Easy21? Why?
•What are the pros and cons of function approximation in Easy21?
•How would you modify the function approximator suggested in this sectionto get better results in Easy21?